In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# first compute pred patches using challenge_conv1d.py, then postprocess with this notebook
data = np.load('tmp/pred_patches.npy', allow_pickle=True).item()

In [ ]:
print(data['casing'].keys())

In [ ]:
# visualize
well = 10
section = 0
patch = 0
patches = []
while True:
    name = f'well_{well}_section_0_patch_{patch}'
    if name in data['tie']:
        patches.append(data['tie'][name])
        patch += 1
    else:
        break
patches = np.concatenate(patches, axis=0)
plt.imshow(patches)

In [ ]:
# visualize score map
import torch
def compute_score_map(img, ball_size=3):
    """Compute score map, assume second dim is time"""
    assert ball_size % 2 == 1, "Ball size must be odd"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    # build the terrain
    timg = torch.from_numpy(img)
    terrain = torch.cat(
        (torch.zeros(1, img.shape[1]), timg, torch.zeros(1, img.shape[1])), dim=0
    ).to(device)
    timg = timg.to(device)
    terrain_down = terrain.clone()
    terrain_up = terrain.clone()
    maxpool = torch.nn.MaxPool1d(ball_size, stride=1, padding=ball_size // 2)
    for row_ind in range(1, terrain.shape[0] - 1):  # forward
        # now the terrain has the max value that can be achieved from top to the current row
        terrain_down[row_ind] = (
            maxpool(terrain_down[row_ind - 1 : row_ind])[0] + terrain_down[row_ind]
        )
    for row_ind in range(terrain.shape[0] - 2, 0, -1):  # backward
        # now the terrain has the max value that can be achieved from bottom to the current row
        terrain_up[row_ind] = (
            maxpool(terrain_up[row_ind + 1 : row_ind + 2])[0] + terrain_up[row_ind]
        )
    terrain = (terrain_up + terrain_down)[
        1:-1
    ] - timg  # remove the padding and the original image$
    terrain = terrain / terrain.shape[0]  # mean node on path
    return terrain.cpu().numpy()

score_map = compute_score_map(patches)
plt.figure()
plt.imshow((score_map == np.max(score_map, axis=1, keepdims=True)) + score_map / score_map.max())

In [ ]:

import pandas as pd

In [ ]:
# inference!
kernel_size = 21
wells = sorted(list(set([name.split('_')[1] for name in data['casing'].keys()])))
predictions = dict()
for well in wells:
    sections = sorted(list(set([name.split('_')[3] for name in data['casing'].keys() if f"well_{well}" in name])))
    for section in sections:
        print(f'well_{well}_section_{section}')
        patch = 0
        casings, ties = [], []
        while True:
            name = f"well_{well}_section_{section}_patch_{patch}"
            if name in data['casing']:
                casing = data['casing'][name]
                tie = data['tie'][name]
                casings.append(casing)
                ties.append(tie)
            else:
                break
            patch += 1
        casings = np.concatenate(casings, axis=0)
        ties = np.concatenate(ties, axis=0)

        casing_score = compute_score_map(casings)
        casing_line = casing_score == np.max(casing_score, axis=1, keepdims=True)
        
        tie_score = compute_score_map(ties)
        tie_line = tie_score == np.max(tie_score, axis=1, keepdims=True)

        out_label = np.clip(casing_line * 2 + tie_line * 1, min=0, max=2)
        out_label = torch.nn.MaxPool2d(kernel_size=(1, kernel_size), stride=1, padding=(0, kernel_size//2))(torch.from_numpy(out_label).float()[None, None])[0, 0]

        for patch_number in range(patch):
            patch_name = f"well_{well}_section_{section}_patch_{patch_number}"
            predictions[patch_name] = out_label[patch_number*160:(patch_number+1)*160, :].numpy().astype(int).flatten()
    

    plt.figure()
    plt.imshow(out_label)
print('saving...')
pd.DataFrame(predictions, dtype="int").T.to_csv('y_test_csv_file_line_21.csv')
print('saved!')

## enforce casing < tie

In [ ]:
def compute_conditional_probabilities_batched(prob_A, prob_B, w=0):
    """
    Compute P(B=b | A + w < B) and P(A=a | A + w < B) for batched probability distributions

    Args:
        prob_A: Array of shape (batch_size, N) containing P(A=a) distributions
        prob_B: Array of shape (batch_size, N) containing P(B=b) distributions
        w: Integer offset (default: 0, which gives A < B condition)

    Returns:
        Tuple of (P(B=b | A + w < B), P(A=a | A + w < B)) both of shape (batch_size, N)
    """
    # Sanity checks
    if prob_A.shape != prob_B.shape:
        raise ValueError("Input arrays must have the same shape")

    batch_size, N = prob_A.shape

    # Check if distributions sum to 1
    sum_A = np.sum(prob_A, axis=1)
    sum_B = np.sum(prob_B, axis=1)
    if not np.all((0.99 <= sum_A) & (sum_A <= 1.01)) or not np.all(
        (0.99 <= sum_B) & (sum_B <= 1.01)
    ):
        raise ValueError("Input arrays must contain valid probability distributions")

    # Compute P(A + w < b) for all b
    # This means A < b - w, so we need P(A < b-w)
    prob_A_less_than_offset = np.zeros_like(prob_A)

    # For each position b, we need the cumulative sum of A up to index b-w-1
    for b in range(N):
        threshold = b - w
        if threshold > 0:
            prob_A_less_than_offset[:, b] = np.sum(prob_A[:, :threshold], axis=1)

    # Compute P(B > a + w) for all a
    prob_B_greater_than_offset = np.zeros_like(prob_B)

    # For each position a, we need the cumulative sum of B from index a+w+1 to the end
    for a in range(N):
        threshold = a + w + 1
        if threshold < N:
            prob_B_greater_than_offset[:, a] = np.sum(prob_B[:, threshold:], axis=1)

    # Compute P(B=b, A + w < B) for all b
    joint_prob_B = prob_B * prob_A_less_than_offset

    # Compute P(A=a, A + w < B) for all a
    joint_prob_A = prob_A * prob_B_greater_than_offset

    # Compute P(A + w < B) = sum of all joint probabilities
    prob_A_plus_w_less_B = np.sum(joint_prob_B, axis=1, keepdims=True)

    # Compute conditional probabilities, handling division by zero
    conditional_prob_B = np.zeros_like(prob_B)
    conditional_prob_A = np.zeros_like(prob_A)

    # Use np.divide with where parameter to handle division by zero
    np.divide(
        joint_prob_B,
        prob_A_plus_w_less_B,
        out=conditional_prob_B,
        where=prob_A_plus_w_less_B > 0,
    )
    np.divide(
        joint_prob_A,
        prob_A_plus_w_less_B,
        out=conditional_prob_A,
        where=prob_A_plus_w_less_B > 0,
    )

    return conditional_prob_B, conditional_prob_A



In [ ]:
# inference!
kernel_size = 21
wells = sorted(list(set([name.split('_')[1] for name in data['casing'].keys()])))
predictions = dict()
for well in wells:
    sections = sorted(list(set([name.split('_')[3] for name in data['casing'].keys() if f"well_{well}" in name])))
    for section in sections:
        print(f'well_{well}_section_{section}')
        patch = 0
        casings, ties = [], []
        while True:
            name = f"well_{well}_section_{section}_patch_{patch}"
            if name in data['casing']:
                casing = data['casing'][name]
                tie = data['tie'][name]
                casings.append(casing)
                ties.append(tie)
            else:
                break
            patch += 1
        casings = np.concatenate(casings, axis=0)
        ties = np.concatenate(ties, axis=0)
        ties, _ = compute_conditional_probabilities_batched(casings, ties, w=40)

        casing_score = compute_score_map(casings)
        casing_line = casing_score == np.max(casing_score, axis=1, keepdims=True)
        
        tie_score = compute_score_map(ties)
        tie_line = tie_score == np.max(tie_score, axis=1, keepdims=True)

        out_label = np.clip(casing_line * 2 + tie_line * 1, min=0, max=2)
        out_label = torch.nn.MaxPool2d(kernel_size=(1, kernel_size), stride=1, padding=(0, kernel_size//2))(torch.from_numpy(out_label).float()[None, None])[0, 0]

        for patch_number in range(patch):
            patch_name = f"well_{well}_section_{section}_patch_{patch_number}"
            predictions[patch_name] = out_label[patch_number*160:(patch_number+1)*160, :].numpy().astype(int).flatten()
    

    plt.figure()
    plt.imshow(out_label)
print('saving...')
pd.DataFrame(predictions, dtype="int").T.to_csv('y_test_csv_file_conditioned_line_21.csv')
print('saved!')

## now enforce cosine regularity

In [ ]:

from cosines import inference

In [ ]:
# inference!
kernel_size = 21
wells = sorted(list(set([name.split('_')[1] for name in data['casing'].keys()])))
predictions = dict()
for well in wells:
    sections = sorted([int(e) for e in set([name.split('_')[3] for name in data['casing'].keys() if f"well_{well}" in name])])
    images = {'tie': [], 'casing': []}
    depth_score_maps_casing = []
    depth_score_maps_tie = []
    for section in sections:
        print(f'well_{well}_section_{section}', end='\r')
        patch = 0
        casings, ties = [], []
        while True:
            name = f"well_{well}_section_{section}_patch_{patch}"
            if name in data['casing']:
                casing = data['casing'][name]
                tie = data['tie'][name]
                casings.append(casing)
                ties.append(tie)
            else:
                break
            patch += 1
        casings = np.concatenate(casings, axis=0)
        ties = np.concatenate(ties, axis=0)
        images['casing'].append(casings)
        images['tie'].append(ties)

    images = {'casing': np.array(images['casing']), 'tie': np.array(images['tie'])}
    A, R, T = images['casing'].shape

    final_score_maps_casing = inference(images['casing'])[0]
    final_score_maps_tie = inference(images['tie'])[0]

    casing_lines = final_score_maps_casing == np.max(final_score_maps_casing, axis=2, keepdims=True)
    tie_lines = final_score_maps_tie == np.max(final_score_maps_tie, axis=2, keepdims=True)
    print(out_label.shape, casing_lines.shape, tie_lines.shape)
    out_label = np.clip(casing_lines * 2 + tie_lines * 1, min=0, max=2)
    out_label = torch.nn.MaxPool2d(kernel_size=(1, kernel_size), stride=1, padding=(0, kernel_size//2))(torch.from_numpy(out_label).float()[:, None])[:, 0]

    for sind, section in enumerate(sections):
        for patch_number in range(patch):
            patch_name = f"well_{well}_section_{section}_patch_{patch_number}"
            predictions[patch_name] = out_label[section][patch_number*160:(patch_number+1)*160, :].numpy().astype(int).flatten()

    plt.figure()
    plt.imshow(out_label[0])
print('saving...')
pd.DataFrame(predictions, dtype="int").T.to_csv('y_test_csv_file_line_21_cosine.csv')
print('saved!')